Feature selection

Genetic Feature Selection

In [1]:
### Redo with 23 generations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn_genetic import *
from sklearn_genetic.space import *
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTEN

pd.set_option("display.max_columns", None)

def myGAFeature(X, y, estimate, flag):
    
    model = lambda aPipe: Pipeline([('scaler',  aPipe), ('estimate', estimate)])
    
    cv = 2 
    top_k = 1
    dispatch = '4*n_jobs'
    return_train = True
    gen = 23
    
    def analyse(model):
        geneSelectFeature = GAFeatureSelectionCV(estimator = model, cv=cv, 
                                                 scoring=None, population_size=50, 
                                                 generations=gen, crossover_probability=0.2, 
                                                 mutation_probability=0.8, tournament_size=3, 
                                                 elitism=True, max_features=None, verbose=True, 
                                                 keep_top_k=top_k, criteria='max', 
                                                 algorithm='eaMuPlusLambda', refit=True, 
                                                 n_jobs=4, 
                                                 pre_dispatch=dispatch, error_score=np.nan, 
                                                 return_train_score=return_train, log_config=None)

        geneSelectFeature.fit(X_train, y_train)
        y_pred = geneSelectFeature.predict(X_test)
        print('best estimator ' + str(geneSelectFeature.best_estimator_))
        print()
        outcome_labels = ['Intubation False', 'Intubation True']
        print('Genetic Feature Selection:', geneSelectFeature.support_, '\n')

        print('classification report \n', classification_report(y_test, y_pred, target_names=outcome_labels))
        micro_roc_auc_ovr = roc_auc_score(y_test, y_pred, multi_class="ovr", average="micro")
        print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")
        print('\n')
        return ""
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state = 0) #train split
    
    if flag == "std":
        print('StandardScaler')
        analyse(model(StandardScaler()))
        
    elif flag == "rbt":
        print('RobustScaler')
        analyse(model(RobustScaler()))
    return ""


print('### Random Forest ###')
print()

print('SMOTEN median noncontin')
print()

print('Genetic Algorithm Feature Selection')
smoten_noncontin = pd.read_csv('smoten_noncontin.csv', index_col=False)


X = smoten_noncontin.drop('outcome',axis= 1)
y = smoten_noncontin['outcome']


best_paramsS =  {'estimate__n_estimators': 150}
best_paramsR =  {'estimate__n_estimators': 50}

estimate = RandomForestClassifier(
                        criterion='entropy', n_estimators= best_paramsS['estimate__n_estimators'],
                        max_depth=None, min_samples_split=2, 
                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                        max_features='sqrt', max_leaf_nodes=None, 
                        min_impurity_decrease=0.0, bootstrap=True, oob_score=True, 
                        n_jobs=4, 
                        random_state=None, verbose=0, warm_start=False, 
                        class_weight=None, ccp_alpha=0.0, max_samples=None)

myGAFeature(X, y, estimate, 'std')

estimate = RandomForestClassifier(
                        criterion='entropy', n_estimators= best_paramsR['estimate__n_estimators'],
                        max_depth=None, min_samples_split=2, 
                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                        max_features='sqrt', max_leaf_nodes=None, 
                        min_impurity_decrease=0.0, bootstrap=True, oob_score=True, 
                        n_jobs=4, 
                        random_state=None, verbose=0, warm_start=False, 
                        class_weight=None, ccp_alpha=0.0, max_samples=None)

myGAFeature(X, y, estimate, 'rbt')


### Random Forest ###

SMOTEN median noncontin

Genetic Algorithm Feature Selection
StandardScaler
gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	50    	0.689189	0.0803762  	0.844905   	0.554376   
1  	100   	0.76255 	0.0433052  	0.844905   	0.697614   
2  	100   	0.767004	0.053641   	0.844905   	0.625053   
3  	100   	0.769176	0.06083    	0.844834   	0.580685   
4  	100   	0.775515	0.0733426  	0.844888   	0.615579   
5  	100   	0.782433	0.0787423  	0.844888   	0.567371   
6  	100   	0.78953 	0.0798372  	0.844888   	0.567371   
7  	100   	0.791758	0.0766122  	0.844888   	0.567371   
8  	100   	0.77861 	0.083864   	0.844905   	0.5633     
9  	100   	0.791181	0.0700968  	0.844888   	0.618067   
10 	100   	0.796291	0.0699924  	0.844888   	0.581414   
11 	100   	0.771861	0.0830091  	0.844888   	0.567371   
12 	100   	0.781315	0.0819867  	0.844977   	0.5633     
13 	100   	0.777569	0.0809452  	0.844977   	0.580685   
14 	100   	0.783129	0.0786133  	0.844977   	0.612717   
15 	1

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(


best estimator Pipeline(steps=[('scaler', StandardScaler()),
                ('estimate',
                 RandomForestClassifier(criterion='entropy', n_estimators=150,
                                        n_jobs=4, oob_score=True))])

Genetic Feature Selection: [ True  True  True  True  True  True] 

classification report 
                   precision    recall  f1-score   support

Intubation False       0.85      0.82      0.84      6902
 Intubation True       0.83      0.87      0.85      7162

        accuracy                           0.84     14064
       macro avg       0.84      0.84      0.84     14064
    weighted avg       0.84      0.84      0.84     14064

Micro-averaged One-vs-Rest ROC AUC score:
0.84


RobustScaler
gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	50    	0.668498	0.0822687  	0.813211   	0.529455   
1  	100   	0.745036	0.0543426  	0.844639   	0.593803   
2  	100   	0.769906	0.0417351  	0.844639   	0.708156   
3  	100   	0.752703	0.0501455  	0

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(


''

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn_genetic import *
from sklearn_genetic.space import *
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTEN

pd.set_option("display.max_columns", None)

def myGAFeature(X, y, estimate, flag):
    
    model = lambda aPipe: Pipeline([('scaler',  aPipe), ('estimate', estimate)])
    
    cv = 2 
    top_k = 1
    dispatch = '4*n_jobs'
    return_train = True
    gen = 23
    
    def analyse(model):
        geneSelectFeature = GAFeatureSelectionCV(estimator = model, cv=cv, 
                                                 scoring=None, population_size=50, 
                                                 generations=gen, crossover_probability=0.2, 
                                                 mutation_probability=0.8, tournament_size=3, 
                                                 elitism=True, max_features=None, verbose=True, 
                                                 keep_top_k=top_k, criteria='max', 
                                                 algorithm='eaMuPlusLambda', refit=True, 
                                                 n_jobs=4, 
                                                 pre_dispatch=dispatch, error_score=np.nan, 
                                                 return_train_score=return_train, log_config=None)

        geneSelectFeature.fit(X_train, y_train)
        y_pred = geneSelectFeature.predict(X_test)
        print('best estimator ' + str(geneSelectFeature.best_estimator_))
        print()
        outcome_labels = ['Intubation False', 'Intubation True']
        print('Genetic Feature Selection:', geneSelectFeature.support_, '\n')

        print('classification report \n', classification_report(y_test, y_pred, target_names=outcome_labels))
        micro_roc_auc_ovr = roc_auc_score(y_test, y_pred, multi_class="ovr", average="micro")
        print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")
        print('\n')
        return ""
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state = 0) #train split
    
    if flag == "std":
        print('StandardScaler')
        analyse(model(StandardScaler()))
        
    elif flag == "rbt":
        print('RobustScaler')
        analyse(model(RobustScaler()))
    return ""


print('### Random Forest ###')
print()

print('SMOTEN knn contin')
print()

print('Genetic Algorithm Feature Selection')
smoten_knn_contin = pd.read_csv('smoten_knn_contin.csv', index_col=False)


X = smoten_knn_contin.drop('outcome',axis= 1)
y = smoten_knn_contin['outcome']


best_paramsS =  {'estimate__n_estimators': 150}
best_paramsR =  {'estimate__n_estimators': 100}

estimate = RandomForestClassifier(
                        criterion='entropy', n_estimators= best_paramsS['estimate__n_estimators'],
                        max_depth=None, min_samples_split=2, 
                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                        max_features='sqrt', max_leaf_nodes=None, 
                        min_impurity_decrease=0.0, bootstrap=True, oob_score=True, 
                        n_jobs=4, 
                        random_state=None, verbose=0, warm_start=False, 
                        class_weight=None, ccp_alpha=0.0, max_samples=None)

myGAFeature(X, y, estimate, 'std')

estimate = RandomForestClassifier(
                        criterion='entropy', n_estimators= best_paramsR['estimate__n_estimators'],
                        max_depth=None, min_samples_split=2, 
                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                        max_features='sqrt', max_leaf_nodes=None, 
                        min_impurity_decrease=0.0, bootstrap=True, oob_score=True, 
                        n_jobs=4, 
                        random_state=None, verbose=0, warm_start=False, 
                        class_weight=None, ccp_alpha=0.0, max_samples=None)

myGAFeature(X, y, estimate, 'rbt')


### Random Forest ###

SMOTEN knn contin

Genetic Algorithm Feature Selection
StandardScaler
gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	50    	0.97764	0.00177522 	0.98002    	0.972323   
1  	100   	0.979189	0.000736982	0.980198   	0.977496   
2  	100   	0.979516	0.000650689	0.980233   	0.977051   
3  	100   	0.979274	0.00114935 	0.980233   	0.975398   
4  	100   	0.979601	0.000889414	0.980358   	0.975807   
5  	100   	0.979537	0.000848357	0.980358   	0.976998   
6  	100   	0.979435	0.00097836 	0.980411   	0.97682    
7  	100   	0.979821	0.000640001	0.980447   	0.977158   
8  	100   	0.979576	0.00107011 	0.980447   	0.975665   
9  	100   	0.979594	0.00111817 	0.980447   	0.976731   
10 	100   	0.979597	0.00117288 	0.980447   	0.975665   
11 	100   	0.97974 	0.0010617  	0.980447   	0.975611   
12 	100   	0.979702	0.0010228  	0.980464   	0.976625   
13 	100   	0.979787	0.00096617 	0.980464   	0.976163   
14 	100   	0.979702	0.00102978 	0.980464   	0.97634    
15 	100   	0.

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(


best estimator Pipeline(steps=[('scaler', StandardScaler()),
                ('estimate',
                 RandomForestClassifier(criterion='entropy', n_estimators=150,
                                        n_jobs=4, oob_score=True))])

Genetic Feature Selection: [ True False False  True False  True False  True  True  True  True  True
  True  True False  True  True  True  True False  True  True False  True
 False  True  True  True] 

classification report 
                   precision    recall  f1-score   support

Intubation False       0.96      1.00      0.98      6902
 Intubation True       1.00      0.96      0.98      7162

        accuracy                           0.98     14064
       macro avg       0.98      0.98      0.98     14064
    weighted avg       0.98      0.98      0.98     14064

Micro-averaged One-vs-Rest ROC AUC score:
0.98


RobustScaler
gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	50    	0.97799	0.00161989 	0.980287   	0.973283   
1  	100   	0.

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(


''

In [1]:
print('SFS Forward Feature Selection')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn_genetic import *
from sklearn_genetic.space import *
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTEN

pd.set_option("display.max_columns", None)

def myGAFeature(X, y, estimate, flag, feature_count):
    
    model = lambda aPipe: Pipeline([('scaler',  aPipe), ('estimate', estimate)])
    
    cv = 2 
    top_k = 1
    dispatch = '4*n_jobs'
    return_train = True
    gen = 22
    
    def analyse(model):
        SelectFeature = SequentialFeatureSelector(estimator=model, n_features_to_select=feature_count, tol=None, direction='forward', 
                                                  scoring='f1_micro', cv=cv, n_jobs=4)

        SelectFeature.fit(X_train, y_train)
        #y_pred = SelectFeature.predict(X_test)
        #print('best estimator ' + str(SelectFeature.best_estimator_))
        print()
        outcome_labels = ['Intubation False', 'Intubation True']
        print('Feature Selection:') 
        features = []
        for i in range(len(SelectFeature.feature_names_in_)):
            if SelectFeature.support_[i] == True:
                features.append(SelectFeature.feature_names_in_[i])
        print(features)
        print()
        #print('classification report \n', classification_report(y_test, y_pred, target_names=outcome_labels))
        #micro_roc_auc_ovr = roc_auc_score(y_test, y_pred, multi_class="ovr", average="micro")
        #print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")
        #print('\n')
        return ""
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state = 0) #train split
    
    if flag == "std":
        print('StandardScaler')
        analyse(model(StandardScaler()))
        
    elif flag == "rbt":
        print('RobustScaler')
        analyse(model(RobustScaler()))
    return ""


print('### Random Forest ###')
print()

print('SMOTEN knn contin')
print()

print('SFS Forward Feature Selection')
smoten_knn_contin = pd.read_csv('smoten_knn_contin.csv', index_col=False).drop(['pt_min','pt_max','urineoutput'],axis=1)


X = smoten_knn_contin.drop('outcome',axis= 1)
y = smoten_knn_contin['outcome']


best_paramsS =  {'estimate__n_estimators': 150}
best_paramsR =  {'estimate__n_estimators': 100}

estimate = RandomForestClassifier(
                        criterion='entropy', n_estimators= best_paramsS['estimate__n_estimators'],
                        max_depth=None, min_samples_split=2, 
                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                        max_features='sqrt', max_leaf_nodes=None, 
                        min_impurity_decrease=0.0, bootstrap=True, oob_score=True, 
                        n_jobs=4, 
                        random_state=None, verbose=0, warm_start=False, 
                        class_weight=None, ccp_alpha=0.0, max_samples=None)

myGAFeature(X, y, estimate, 'std',5)

estimate = RandomForestClassifier(
                        criterion='entropy', n_estimators= best_paramsR['estimate__n_estimators'],
                        max_depth=None, min_samples_split=2, 
                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                        max_features='sqrt', max_leaf_nodes=None, 
                        min_impurity_decrease=0.0, bootstrap=True, oob_score=True, 
                        n_jobs=4, 
                        random_state=None, verbose=0, warm_start=False, 
                        class_weight=None, ccp_alpha=0.0, max_samples=None)

myGAFeature(X, y, estimate, 'rbt',5)


print('### Random Forest ###')
print()

print('SMOTEN median contin')
print()

print('SFS Forward Feature Selection')
smoten_median_imputed_contin = pd.read_csv('smoten_median_imputed_contin.csv', index_col=False).drop(['pt_min','pt_max','urineoutput'],axis=1)

X = smoten_median_imputed_contin.drop('outcome',axis= 1)
y = smoten_median_imputed_contin['outcome']

best_paramsS =  {'estimate__n_estimators': 100}
best_paramsR =  {'estimate__n_estimators': 150}

estimate = RandomForestClassifier(
                        criterion='entropy', n_estimators= best_paramsS['estimate__n_estimators'],
                        max_depth=None, min_samples_split=2, 
                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                        max_features='sqrt', max_leaf_nodes=None, 
                        min_impurity_decrease=0.0, bootstrap=True, oob_score=True, 
                        n_jobs=4, 
                        random_state=None, verbose=0, warm_start=False, 
                        class_weight=None, ccp_alpha=0.0, max_samples=None)

myGAFeature(X, y, estimate, 'std',5)

estimate = RandomForestClassifier(
                        criterion='entropy', n_estimators= best_paramsR['estimate__n_estimators'],
                        max_depth=None, min_samples_split=2, 
                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                        max_features='sqrt', max_leaf_nodes=None, 
                        min_impurity_decrease=0.0, bootstrap=True, oob_score=True, 
                        n_jobs=4, 
                        random_state=None, verbose=0, warm_start=False, 
                        class_weight=None, ccp_alpha=0.0, max_samples=None)

myGAFeature(X, y, estimate, 'rbt',5)


SFS Forward Feature Selection
### Random Forest ###

SMOTEN knn contin

SFS Forward Feature Selection
StandardScaler

Feature Selection:
['age', 'dbp_mean', 'temperature_mean', 'wbc_max', 'creatinine_min']

RobustScaler

Feature Selection:
['age', 'dbp_mean', 'temperature_mean', 'wbc_max', 'creatinine_min']

### Random Forest ###

SMOTEN median contin

SFS Forward Feature Selection
StandardScaler

Feature Selection:
['age', 'dbp_mean', 'temperature_mean', 'wbc_max', 'creatinine_min']

RobustScaler

Feature Selection:
['age', 'dbp_mean', 'temperature_mean', 'wbc_max', 'creatinine_min']



''

In [3]:
print('### Random Forest ###')
print()

print('SMOTEN median contin')
print()

print('Genetic Algorithm Feature Selection')
smoten_median_imputed_contin = pd.read_csv('smoten_median_imputed_contin.csv', index_col=False)

X = smoten_median_imputed_contin.drop('outcome',axis= 1)
y = smoten_median_imputed_contin['outcome']

best_paramsS =  {'estimate__n_estimators': 100}
best_paramsR =  {'estimate__n_estimators': 150}

estimate = RandomForestClassifier(
                        criterion='entropy', n_estimators= best_paramsS['estimate__n_estimators'],
                        max_depth=None, min_samples_split=2, 
                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                        max_features='sqrt', max_leaf_nodes=None, 
                        min_impurity_decrease=0.0, bootstrap=True, oob_score=True, 
                        n_jobs=4, 
                        random_state=None, verbose=0, warm_start=False, 
                        class_weight=None, ccp_alpha=0.0, max_samples=None)

myGAFeature(X, y, estimate, 'std')

estimate = RandomForestClassifier(
                        criterion='entropy', n_estimators= best_paramsR['estimate__n_estimators'],
                        max_depth=None, min_samples_split=2, 
                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                        max_features='sqrt', max_leaf_nodes=None, 
                        min_impurity_decrease=0.0, bootstrap=True, oob_score=True, 
                        n_jobs=4, 
                        random_state=None, verbose=0, warm_start=False, 
                        class_weight=None, ccp_alpha=0.0, max_samples=None)

myGAFeature(X, y, estimate, 'rbt')


### Random Forest ###

SMOTEN median contin

Genetic Algorithm Feature Selection
StandardScaler
gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	50    	0.977881	0.00153123 	0.980269   	0.973585   
1  	100   	0.979056	0.000859738	0.980322   	0.976251   
2  	100   	0.979338	0.000726394	0.980215   	0.977282   
3  	100   	0.979408	0.000742914	0.980215   	0.976696   
4  	100   	0.979404	0.00109638 	0.980251   	0.975292   
5  	100   	0.979253	0.0011422  	0.980251   	0.97618    
6  	100   	0.979189	0.00110375 	0.980269   	0.975789   
7  	100   	0.97936 	0.00114041 	0.980269   	0.975576   
8  	100   	0.979782	0.000842389	0.980358   	0.976945   
9  	100   	0.979546	0.00108811 	0.980393   	0.976625   
10 	100   	0.979533	0.00137103 	0.980464   	0.973301   
11 	100   	0.979808	0.000804762	0.980447   	0.977122   
12 	100   	0.979476	0.00116058 	0.980464   	0.976323   
13 	100   	0.979507	0.00124716 	0.980464   	0.976287   
14 	100   	0.979172	0.00156991 	0.980464   	0.973834   
15 	100 

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(


best estimator Pipeline(steps=[('scaler', StandardScaler()),
                ('estimate',
                 RandomForestClassifier(criterion='entropy', n_jobs=4,
                                        oob_score=True))])

Genetic Feature Selection: [ True False  True  True  True False  True False  True False False  True
  True  True  True  True  True  True False  True  True  True  True  True
  True  True  True False] 

classification report 
                   precision    recall  f1-score   support

Intubation False       0.96      1.00      0.98      6902
 Intubation True       1.00      0.96      0.98      7162

        accuracy                           0.98     14064
       macro avg       0.98      0.98      0.98     14064
    weighted avg       0.98      0.98      0.98     14064

Micro-averaged One-vs-Rest ROC AUC score:
0.98


RobustScaler
gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	50    	0.977758	0.00133528 	0.980144   	0.974367   
1  	100   	0.979099	0.0008406

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(


''

In [5]:
print('### Random Forest ###')
print()

print('Random Forest all under 40 missing')
print()

print('Genetic Algorithm Feature Selection')
smoten_median_imputed_less_40 = pd.read_csv('smoten_median_imputed_less_40.csv', index_col=False)


X = smoten_median_imputed_less_40.drop('outcome',axis= 1)
y = smoten_median_imputed_less_40['outcome']

best_paramsS =  {'estimate__n_estimators': 150}
best_paramsR =  {'estimate__n_estimators': 50}

estimate = RandomForestClassifier(
                        criterion='entropy', n_estimators= best_paramsS['estimate__n_estimators'],
                        max_depth=None, min_samples_split=2, 
                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                        max_features='sqrt', max_leaf_nodes=None, 
                        min_impurity_decrease=0.0, bootstrap=True, oob_score=True, 
                        n_jobs=4, 
                        random_state=None, verbose=0, warm_start=False, 
                        class_weight=None, ccp_alpha=0.0, max_samples=None)

myGAFeature(X, y, estimate, 'std')

estimate = RandomForestClassifier(
                        criterion='entropy', n_estimators= best_paramsR['estimate__n_estimators'],
                        max_depth=None, min_samples_split=2, 
                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                        max_features='sqrt', max_leaf_nodes=None, 
                        min_impurity_decrease=0.0, bootstrap=True, oob_score=True, 
                        n_jobs=4, 
                        random_state=None, verbose=0, warm_start=False, 
                        class_weight=None, ccp_alpha=0.0, max_samples=None)

myGAFeature(X, y, estimate, 'rbt')


### Random Forest ###

Random Forest all under 40 missing

Genetic Algorithm Feature Selection
StandardScaler
gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	50    	0.97868	0.00102255 	0.980589   	0.975505   
1  	100   	0.979393	0.000855827	0.980589   	0.977247   
2  	100   	0.979672	0.000630161	0.980589   	0.978296   
3  	100   	0.979536	0.000795808	0.980589   	0.977282   
4  	100   	0.979779	0.000596021	0.980589   	0.978207   
5  	100   	0.979945	0.0004636  	0.980589   	0.978847   
6  	100   	0.979775	0.000708667	0.980589   	0.977958   
7  	100   	0.979902	0.000622082	0.980589   	0.976909   
8  	100   	0.979742	0.000847765	0.980589   	0.977656   
9  	100   	0.979872	0.00097565 	0.980589   	0.97458    
10 	100   	0.979832	0.000736507	0.980695   	0.977318   
11 	100   	0.97954 	0.00101262 	0.980589   	0.977247   
12 	100   	0.979879	0.000778988	0.980589   	0.977336   
13 	100   	0.979755	0.00090504 	0.980535   	0.976767   
14 	100   	0.979822	0.000811126	0.980553   	0.977265

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(


best estimator Pipeline(steps=[('scaler', StandardScaler()),
                ('estimate',
                 RandomForestClassifier(criterion='entropy', n_estimators=150,
                                        n_jobs=4, oob_score=True))])

Genetic Feature Selection: [ True  True  True False False  True  True  True  True  True False False
 False  True  True False  True False  True  True  True  True  True  True
  True  True False  True  True False  True  True  True] 

classification report 
                   precision    recall  f1-score   support

Intubation False       0.96      1.00      0.98      6902
 Intubation True       1.00      0.96      0.98      7162

        accuracy                           0.98     14064
       macro avg       0.98      0.98      0.98     14064
    weighted avg       0.98      0.98      0.98     14064

Micro-averaged One-vs-Rest ROC AUC score:
0.98


RobustScaler
gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	50    	0.978004	0.00142781 	0.980

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(


''